# Занятие на кафедре ММП ВМК МГУ. Итераторы и генераторы

Подготовил Александр Чернышёв.

## Итераторы

### Цикл `for`

Можно использовать, чтобы пробежаться по списку:

In [1]:
for i in [1, 2, 3, 4]:
    print(i, end=' ')

1 2 3 4 

Можно использовать, чтобы пробежаться по строке:

In [2]:
for ch in "python":
    print(ch, end=' ')

p y t h o n 

Можно использовать, чтобы пробежаться по ключам словаря:

In [3]:
for k in {"x": 1, "y": 2}:
    print(k, end=' ')

x y 

Можно использовать, чтобы пробежаться по строкам файла:

In [4]:
with open("a.txt", "w") as f:
    print("Hello, world!", "Cat and dog.",
          "Python 3.9.", sep='\n', file=f)

with open("a.txt", "r") as f:
    for line in f:
        print(line, end="")

Hello, world!
Cat and dog.
Python 3.9.


Такие объекты называются итерируемыми (`iterable`).

Многие функции в Python принимают на вход итерируемые объекты:

In [5]:
", ".join(["a", "b", "c"])

'a, b, c'

In [6]:
", ".join({"x": 1, "y": 2})

'x, y'

In [7]:
list("python")

['p', 'y', 't', 'h', 'o', 'n']

In [8]:
tuple({"x": 1, "y": 2})

('x', 'y')

In [9]:
sum([1, 2, 3])

6

In [10]:
all((True, False, True, True))

False

### Протокол итерации

Встроенная функция `iter` принимает на вход итерируемый объект и возвращает итератор.

In [11]:
lst = [1, 2, 3]
x = iter(lst)  # Под капотом: x = lst.__iter__()
x

In [12]:
next(x)  # Под капотом: x.__next__()

1

In [13]:
next(x)

2

In [14]:
next(x)

3

In [15]:
next(x)

StopIteration: 

Каждый раз, когда мы вызываем функцию `next` от итератора, она выдает нам следующий элемент. Если элементов в итераторе больше нет, то он поднимает исключение `StopIteration`.

Итераторы реализованы в виде классов. Вот пример итератора, который работает так же, как встроенный `range` (за исключением меньшей функциональности).

In [16]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [17]:
class yrange:
    def __init__(self, n):
        self.i = 0
        self.n = n

    def __iter__(self):
        return self

    def __next__(self):
        if self.i >= self.n:
            raise StopIteration()

        i = self.i
        self.i += 1
        return i

Метод `__iter__` — это то, что делает объекты итерируемыми. Вызов функции `iter` от объекта просто вызывает его метод `__iter__`.

Метод `__iter__` возвращает итератор. Итератор должен иметь метод `__next__` и поднимать исключение `StopIteration`, когда в итераторе больше не осталось элементов.

Из-за такой специфики работы метода `__next__` по объектам итератора можно пробежаться только 1 раз. После первого прохода по всем его элементам далее он просто будет каждый раз поднимать исключение `StopIteration`.

In [18]:
y = yrange(3)
next(y)

0

In [19]:
next(y)

1

In [20]:
next(y)

2

In [21]:
next(y)

StopIteration: 

Многие встроенные функции принимают итерируемые объекты в качестве аргументов:

In [22]:
list(yrange(5))

[0, 1, 2, 3, 4]

In [23]:
sum(yrange(5))

10

В приведенном выше случае итерируемое и итератор являются одним и тем же объектом, потому что метод `__iter__` возвращает `self`. Не обязательно так делать всегда:

In [24]:
class zrange:
    """Реализация zrange, имитирующего range"""

    def __init__(self, n):
        self.n = n

    def __iter__(self):
        return zrange_iter(self.n)


class zrange_iter:
    """Реализация итератора для zrange"""

    def __init__(self, n):
        self.i = 0
        self.n = n

    def __iter__(self):
        # Iterators are iterables too.
        # Adding this functions to make them so.
        return self

    def __next__(self):
        if self.i < self.n:
            i = self.i
            self.i += 1
            return i
        else:
            raise StopIteration()

Так как итератор поглащается за один проход (см. выше, почему), то в случае, когда итерируемое и итератор являются одним и тем же объектом, то и итерируемое поглощается за один проход:

In [25]:
rng = yrange(3)

list(rng)

[0, 1, 2]

_Небольшое лирическое отступление._

Что под капотом, когда мы вызываем list(rng):

```python
res = []

it = iter(rng)
while True:
    try:
        x = next(it)
    except StopIteration:
        break
    res.append(x)
res
```

Получается, вызов ```list()``` от итерируемого объекта сначала получает итератор на этот объект, после чего достает все элементы из этого итератора и складывает их в список. То есть вызов `list()` от объекта поглащает все элементы из его итератора.

In [26]:
rng = yrange(3)
print(list(rng))

[0, 1, 2]


In [27]:
rng.i, rng.n

(3, 3)

In [28]:
print(list(rng))

[]


Эту проблему можно исправить, например, переместив зануление атрибута `i` в метод `__iter__`:

In [29]:
class yrange:
    def __init__(self, n):
        self.n = n

    def __iter__(self):
        self.i = 0
        return self

    def __next__(self):
        if self.i >= self.n:
            raise StopIteration()

        i = self.i
        self.i += 1
        return i

In [30]:
rng = yrange(3)
list(rng)

[0, 1, 2]

In [31]:
list(rng)

[0, 1, 2]

Но у такого подхода в принципе есть проблема с невозможностью создать одновременно несколько корректных итераторов для объекта, потому что сам объект и является итератором для себя.

Создадим первый итератор:

In [32]:
rng = yrange(3)
it1 = iter(rng)

In [33]:
next(it1)

0

In [34]:
next(it1)

1

Создадим второй итератор:

In [35]:
it2 = iter(rng)

In [36]:
next(it1)

0

Первый итератор обнулился!

Все из-за того, что первый и второй итераторы суть один и тот же объект:

In [37]:
it1 is it2

True

`zrange` не является итератором, поэтому он не будет никогда истощаться:

In [38]:
rng = zrange(3)
print(list(rng))
print(list(rng))

[0, 1, 2]
[0, 1, 2]


А итератор для него, как писалось выше, истощаться будет:

In [39]:
it = iter(rng)
list(it)

[0, 1, 2]

In [40]:
list(it)

[]

В таком подходе проблемы с созданием одновременно нескольких итераторов на объект нет, потому что реализации класса объекта и итератора для него разделены.

In [41]:
rng = zrange(3)
it1 = iter(rng)

In [42]:
next(it1)

0

In [43]:
next(it1)

1

In [44]:
it2 = iter(rng)

In [45]:
next(it1)

2

In [46]:
it1 is it2

False

### Итого
Протокол итераций (протокол итераторов, iterator protocol) заключается в концепции итерирования по контейнерам с помощью методов `__iter__` и `__next__`.

Итерируемое (iterable) — то, что имеет метод `__iter__`.

Итератор (iterator) — то, что имеет методы `__iter__` (должен возвращать `self`) и `__next__`, т.е. поддреживает протокол итераций.

### Обратно к циклу `for`

```python
for elem in elem_collection:
    do_something(elem)
```

Концептуально такой цикл `for` делает следующее:

```python
it = iter(elem_collection) # инициализация
while True:
    try:
        elem = next(it) # получить следующий элемент
    except StopIteration: # если элементов больше нет
        break
    do_something(elem)
```

### Особенности `iter` и `next`

Другая форма вызова `iter`: принимает функцию и значение, вызывает функцию, пока она не вернёт это значение:

In [47]:
import random

random.seed(179)

for x in iter(lambda: random.randint(0, 9), 7):
    print(x, end=' ')

3 1 1 0 8 4 

Для функции `next` можно вторым аргументом указать значение, которое необходимо вернуть в случае исключения:

In [48]:
next(iter([]), 153)

153

In [49]:
next(iter([]))

StopIteration: 

### Протокол итераций и `__getitem__`
Напоминание: `__getitem__` возвращает элемент по индексу либо поднимает `IndexError`, если элемента нет.

In [50]:
lst = [1, 2, 3]
lst[1]  # Под капотом: lst.__getitem__(1)

2

Если реализован метод `__getitem__`, протокол итераций будет реализован автоматически:

In [51]:
class MyList:
    def __init__(self, *args):
        self.list = list(args)

    def __getitem__(self, i):
        return self.list[i]

In [52]:
for elem in MyList(1, 2):
    print(elem, end=' ')

1 2 

In [53]:
iter(MyList(1, 2, 3))

Как реализован протокол итераций исходя из примера?

Примерно следующим образом. Обратите внимание, метод `__getitem__` я переименовал в `getitem`, чтобы не сложилось впечатление, что за нас всю работу сделал `Python`.

В реализации мы создали дополнительный класс для итератора по объектам класса `MyList`. Вообще говоря, написанный итератор подходит для любого объекта, в котором реализован метод `getitem`.

In [54]:
class MyList:
    def __init__(self, *args):
        self.list = list(args)

    def getitem(self, i):
        return self.list[i]

    def __iter__(self):
        return MyListIterator(self)


class MyListIterator:
    def __init__(self, obj):
        self.i = 0
        self.obj = obj

    def __iter__(self):
        return self

    def __next__(self):
        try:
            ret = self.obj.getitem(self.i)
        except IndexError:
            raise StopIteration
        self.i += 1
        return ret

In [55]:
for elem in MyList(1, 2):
    print(elem, end=' ')

1 2 

In [56]:
lst = MyList(1, 2, 3, 4)

In [57]:
it = iter(lst)

In [58]:
it.obj is lst

True

In [59]:
it

### Протокол итераций и `__contains__`
Напоминание: `__contains__` возвращает `True` если переданный элемент содержится в экземпляре (перегрузка `in` и `not in`).

Если класс реализует протокол итераций, метод `__contains__` реализуется автоматически следующим образом:

```python
class SomeClass:
    # это реализуется автоматически
    def __contains__(self, target_elem):
        for elem in self:
            if target_elem == elem:
                return True
        return False
```

Пример для `MyList`:

In [60]:
2 in MyList(1, 2, 3, 4)

True

In [61]:
123 in MyList(1, 2, 3, 4)

False

#### Задача 1

Напишите класс итератора `my_reversed` (аналог `reversed`), который принимает на вход __список__ и выдает его в обратном направлении.

In [62]:
list(reversed([1, 2, 3, 4]))

[4, 3, 2, 1]

Кстати, в чем отличие `reversed([1, 2, 3, 4])` от `[1, 2, 3, 4][::-1]`?

В первом случае не выделяется дополнительная память под еще один список (мы получаем в итоге итератор), во втором случае — выделяется (мы получаем в итоге список).

По сути в этом и заключается прелесть итераторов — они позволяют модифицировать данные "на лету", без выделения дополнительной памяти.

Пример решения:

In [63]:
class my_reversed:
    def __init__(self, lst):
        self.lst = lst
        self.i = len(lst) - 1

    def __iter__(self):
        return self

    def __next__(self):
        if self.i < 0:
            raise StopIteration
        ret = self.lst[self.i]
        self.i -= 1
        return ret

In [64]:
it = my_reversed([1, 2, 3, 4])
list(it)

[4, 3, 2, 1]

In [65]:
for x in my_reversed([1, 2, 3, 4]):
    print(x, end=' ')

4 3 2 1 

In [66]:
it = my_reversed([1, 2, 3, 4])
next(it), next(it), next(it), next(it)

(4, 3, 2, 1)

In [67]:
next(it)

StopIteration: 

Аналогично можно сравнить по используемой памяти следующие куски кода:

In [81]:
import numpy as np

In [82]:
s = 0.0
for x in np.arange(10 ** 7):
    s += x

In [83]:
s = 0
for x in range(10 ** 7):
    s += x

В случае с использованием `numpy.arange` выделяется дополнительная память.

Интереснее сравнить эти примеры по времени работы.

In [71]:
%%timeit

s = 0.0
for x in np.arange(10 ** 7):
    s += x

1.3 s ± 49.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [72]:
%%timeit

s = 0
for x in range(10 ** 7):
    s += x

330 ms ± 4.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
%%timeit

s = sum(range(10 ** 7))

245 ms ± 6.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Как видим, в первом случае выделение памяти и последующая итерация по ячейкам оказывается в 3-4 раза медленнее, чем итерация по итератору `range`.

Еще интересно, что встроенная функция `sum` работает быстрее цикла примерно на 34%.

In [80]:
%%timeit

s = np.arange(10 ** 7).astype(float).sum()

51 ms ± 585 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Правда, использование встроенной функции для суммирования в `numpy` оказывается в 5-6 раз быстрее, чем реализация на чистом `python`. Это обсуловлено тем, что операции в `numpy` реализованы на `C/C++`, из-за чего они работают быстрее питоновских (то есть, получается, цикл на `C/C++` работает примерно в 25 раз быстрее, чем цикл на `python`).

In [79]:
%%timeit

n = 10 ** 7
s = n * (n - 1) // 2

82.1 ns ± 1.22 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Ну и не стоит забывать, что иногда можно вывести аналитическую формулу для ответа и тогда вычисления могут стать гораздо быстрее (но это не всегда работает).

---

## Генераторы

Генераторы — более простой способ создания итераторов. Генератор — это функция, которая выдает последовательность результатов вместо одного значения.

In [84]:
def yrange(n):
    i = 0
    while i < n:
        yield i
        i += 1

In [85]:
y = yrange(3)
y

<generator object yrange at 0x000002096B48C190>

In [86]:
next(y)

0

In [87]:
next(y)

1

In [88]:
next(y)

2

In [89]:
next(y)

StopIteration: 

Генератор также является итератором. Вам не нужно беспокоиться о протоколе итераций.

Слово “генератор” ошибочно используется как для обозначения функции, которая генерирует, так и для того, что она генерирует. Будем использовать слово “генератор” для обозначения созданного генератором объекта, а “функция-генератор” — для обозначения функции, которая его генерирует.

Когда вызывается функция-генератор, она возвращает генератор, даже не начиная выполнение функции. Когда `next` вызывается в первый раз, функция начинает выполняться до тех пор, пока не достигнет оператора `yield`. Полученное значение возвращается на вызов `next`.

Следующий пример демонстрирует взаимодействие между `yield` и вызовом метода `__next__` у генератора:

In [90]:
def foo():
    print("begin")
    for i in range(3):
        print("before yield", i)
        yield i
        print("after yield", i)
    print("end")

In [91]:
f = foo()

In [92]:
next(f)

begin
before yield 0


0

In [93]:
next(f)

after yield 0
before yield 1


1

In [94]:
next(f)

after yield 1
before yield 2


2

In [95]:
next(f)

after yield 2
end


StopIteration: 

Рассмотрим пример:

In [96]:
def integers():
    """Infinite sequence of integers."""
    i = 1
    while True:
        yield i
        i = i + 1


def squares():
    for i in integers():
        yield i * i


def take(n, seq):
    """Returns first n values from the given sequence."""
    seq = iter(seq)
    result = []
    try:
        for i in range(n):
            result.append(next(seq))
    except StopIteration:
        pass
    return result

In [97]:
take(10, integers())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [98]:
take(5, squares())

[1, 4, 9, 16, 25]

In [ ]:
# Если запустите, то получите длинное полотно вывода...
for x in integers():
    print(x)

Еще пример.

Генератор `unique` возвращает из итерируемого объекта только уникальные значения в нем.

In [99]:
def unique(iterable, seen=None):
    seen = set(seen or [])
    for item in iterable:
        if item not in seen:
            seen.add(item)
            yield item

xs = [1, 1, 2, 3]
unique(xs)

<generator object unique at 0x000002096B48C970>

In [100]:
list(unique(xs))

[1, 2, 3]

In [101]:
list(unique(xs, [2]))

[1, 3]

Еще пример.

Генератор `chain` объединяет несколько итерируемых объектов в один.

In [102]:
def chain(*iterables):
    for iterable in iterables:
        for item in iterable:
            yield item

xs = range(3)
ys = [42]
chain(xs, ys)

<generator object chain at 0x000002096B48CB30>

In [103]:
list(chain(xs, ys))

[0, 1, 2, 42]

In [104]:
42 in chain(xs, ys)

True

### Истощение

Генераторы, как и любые итераторы, истощаются:

In [105]:
def f():
    yield 42

gen = f()
list(gen)

[42]

In [106]:
list(gen)

[]

Не надо переиспользовать генераторы!

### Генераторные выражения

Генераторное выражение — это генераторная версия генератора списков (list comprehensions). Генераторные выражения выглядят как генераторы списков, но вместо списка возвращают генератор.

In [107]:
gen = (x ** 2 for x in range(0, 5))
gen

<generator object <genexpr> at 0x000002096B48CF20>

In [108]:
sum(gen)

30

In [110]:
list(gen)  # Генераторные выражения тоже истощаются

[]

In [111]:
gen = (x ** 2 for x in range(0, 5))
list(gen)

[0, 1, 4, 9, 16]

In [112]:
for x in (x ** 2 for x in range(0, 5)):
    print(x, end=' ')

0 1 4 9 16 

Генераторы списков, словарей множеств не являются генераторами!

### `yield from`

Оператор `yield from` позволяет делегировать выполнение другому генератору:

In [113]:
def f():
    yield 1
    yield 2

def g():
    yield 4
    yield 12
    yield 25

def h():
    yield from f()
    yield from g()
    yield from f()

list(h())

[1, 2, 4, 12, 25, 1, 2]

---

## Стандартные итераторы

### `range`

`range` возвращает итерируемый объект `range object` (не итератор!).

`range` при любых аргументах требует константный объём памяти.

In [114]:
my_range = range(0, 5)
my_range

range(0, 5)

In [115]:
next(my_range)

TypeError: 'range' object is not an iterator

In [116]:
iter(my_range)

In [117]:
next(iter(my_range))

0

In [118]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### `zip`

`zip` возвращает итератор.

In [119]:
my_list_1 = [1, 2]
my_list_2 = ['a', 'b']
my_tuples = zip(my_list_1, my_list_2)
my_tuples

In [120]:
iter(my_tuples)

In [121]:
next(my_tuples)

(1, 'a')

In [122]:
next(my_tuples)

(2, 'b')

In [123]:
next(my_tuples)

StopIteration: 

### `enumerate`

`enumerate` возвращает итератор

In [124]:
my_list = ['a', 'b', 'c']
it = enumerate(my_list)
it

In [125]:
next(it)

(0, 'a')

### `map`

`map(func, sequence1, sequence2 ... )` возвращает итератор, состоящий из результатов применений функции `func` к элементам `sequence1`, `sequence2`, ....

* `func` — функция, которая применяется к каждому элементу `sequence`
* `sequence1`, `sequence2` ... — итераторы

In [126]:
it = map(lambda x: x * 2, [0, 1, 2, 3])
list(it)

[0, 2, 4, 6]

In [127]:
it = map(lambda x, y: x + y, [0, 1, 2, 3], [10, 100, 500])
list(it)

[10, 101, 502]

### `filter`
`filter(func, sequence)` возвращает итератор, состоящий из элементов `sequence`, для которых `func` вернула `True`

In [128]:
it = filter(lambda x: x > 0, [0, -1, 2, -3, 5])
list(it)

[2, 5]

И `filter`, и `map` можно проще записать через генераторы списков!

### `reduce`
`reduce(func, sequence)` — кумулятивное применение функции `func` к элементам последовательности `sequence`.

In [129]:
from functools import reduce

res = reduce(lambda x, y: x * y, [1, 2, 3, 4])
res

24

In [130]:
res = reduce(lambda x, y: '({}*{})'.format(str(x), str(y)),
             [1, 2, 3, 4])
res

'(((1*2)*3)*4)'

### Соединение и повторение итераторов
Написанная нами `chain` есть в `itertools`:

In [131]:
from itertools import chain

it1 = iter([1, 2, 3])
it2 = iter([4, 5])
list(chain(it1, it2))

[1, 2, 3, 4, 5]

`repeat` для создания итератора повторяющейся последовательности:

In [132]:
from itertools import repeat

list(repeat([1, 2], 4))

[[1, 2], [1, 2], [1, 2], [1, 2]]

### Бесконечные итераторы

In [133]:
from itertools import count # бесконечный range

c = []
for i in count(0, 5):
    c.append(i)
    if i > 20:
        break
c

[0, 5, 10, 15, 20, 25]

In [134]:
from itertools import cycle # бесконечный повтор последовательности

d = []
it = cycle([1, 2, 3])
for i, j in enumerate(it):
    if i > 6:
        break
    d.append(j)
print(d)

[1, 2, 3, 1, 2, 3, 1]


### Срезы и взятие частей
Срез для произвольного итератора:

In [135]:
from itertools import islice

e = islice(range(10), 0, 8, 2)
list(e)

[0, 2, 4, 6]

Выбрасывать из итератора элементы, пока не найдётся элемент, удовлетворяющий условию:

In [136]:
from itertools import dropwhile

f = dropwhile(lambda x: x < 5, range(10))
list(f)

[5, 6, 7, 8, 9]

### Комбинаторные итераторы

In [137]:
from itertools import permutations

it = permutations("123")
for x in it:
    print(''.join(x))

123
132
213
231
312
321


In [138]:
from itertools import combinations

it = combinations("ABC", 2)
for x in it:
    print(''.join(x))

AB
AC
BC


In [139]:
from itertools import product

it = product("AB", repeat=2)
for x in it:
    print(''.join(x))

AA
AB
BA
BB


### Задача 2

Написать `permutations`, `combinations`, `product`.

TBD...

## Итераторы в задачах с данными большого объёма

Итераторы могут быть полезны в ситуациях, когда хранить всю выборку в памяти невозможно.

Алгоритм работы:

1. Сохранить данные на жёстком диске в хорошем формате;
2. Написать итератор, считывающий данные неполностью и преобразующий их в вектора;
3. Обучать алгоритм по батчам данных.

## Варианты применения итераторов в data science
* Работа с огромной текстовой коллекцией (коллекцией изображений, и т.д.);
* Изменение порядка подачи объектов на обучение;
* Быстрая генерация из сложных объектов более простых (например, по тексту генерируем пары предложений).

---

## Заключение
* Итераторы позволяют не хранить одновременно в памяти все значения последовательности;
* Итераторы можно задавать с помощью описания специальных методов `__iter__` и `__next__` или декларативно с помощью генераторов;
* Многие операции с операторами уже реализованы в библиотеке `itertools` либо средствами `itertools` их можно реализовать проще;
* Итераторы необходимо применять тогда, когда невозможно хранить весь объём данных в памяти.